In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    SparkSession.builder.appName("Desafio - pergunta 1 - Spark SQL")
    .config('spark.sql.repl.eagerEval.enabled', True)
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)

In [4]:
train = spark.read.csv("../datasets/titanic/train.csv", header=True, inferSchema=True)
train

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22.0,1,0,A/5 21171,7.25,NULL,S
2,1,1,"Cumings, Mrs. Joh...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26.0,0,0,STON/O2. 3101282,7.925,NULL,S
4,1,1,"Futrelle, Mrs. Ja...",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35.0,0,0,373450,8.05,NULL,S
6,0,3,"Moran, Mr. James",male,NULL,0,0,330877,8.4583,NULL,Q
7,0,1,"McCarthy, Mr. Tim...",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. ...",male,2.0,3,1,349909,21.075,NULL,S
9,1,3,"Johnson, Mrs. Osc...",female,27.0,0,2,347742,11.1333,NULL,S
10,1,2,"Nasser, Mrs. Nich...",female,14.0,1,0,237736,30.0708,NULL,C


# Questão 1

In [5]:
total_passageiros = train.agg({'PassengerId': 'count'})
total_passageiros

count(PassengerId)
891


In [6]:
sobreviventes = train.where(F.col("Survived") == 1).count()
sobreviventes

342

In [7]:
percentual_sobreviventes = (sobreviventes / total_passageiros.collect()[0][0]) * 100
percentual_sobreviventes

38.38383838383838

In [8]:
df_train = train.createOrReplaceTempView("train")
spark.sql(
    '''
    SELECT 
        total_sobreviventes * 100 / total_passageiros AS percentual_sobreviventes
        FROM (
        SELECT 
            COUNT(PassengerId) AS total_passageiros,
            SUM(CASE WHEN Survived = 1 THEN 1 ELSE 0 END) AS total_sobreviventes
        FROM train
        )
    '''
)

percentual_sobreviventes
38.38383838383838


# Questão 2

In [9]:
spark.sql (
    '''
    SELECT TOTAL_IDADE / nao_sobreviventes AS media_idade_nao_sobreviventes
    FROM (
    SELECT 
        SUM(CASE WHEN Survived = 0 AND Age IS NOT NULL THEN 1 ELSE 0 END) AS nao_sobreviventes,
        SUM(CASE WHEN Survived = 0 AND Age IS NOT NULL THEN Age ELSE 0 END) AS TOTAL_IDADE
    FROM train
    )
    '''
)


media_idade_nao_sobreviventes
30.62617924528302


# Questão 3

In [12]:
spark.sql(
    '''
    SELECT 
        SUM(CASE WHEN Survived = 1 AND Pclass = 3 THEN 1 ELSE 0 END) AS total_sobreviventes_3classe
    FROM train
    '''
)

total_sobreviventes_3classe
119


# Questão 4

In [13]:
spark.sql(
    '''  
    SELECT 
        SUM(CASE WHEN Sex = 'female' AND Survived = 1 THEN 1 ELSE 0 END) * 100 / SUM(CASE WHEN Sex = 'female' THEN 1 ELSE 0 END)
    FROM train
        '''
)

((sum(CASE WHEN ((Sex = female) AND (Survived = 1)) THEN 1 ELSE 0 END) * 100) / sum(CASE WHEN (Sex = female) THEN 1 ELSE 0 END))
74.20382165605096


# Questão 5

In [19]:
spark.sql(
    '''
    SELECT 
        AVG(fare)
    FROM train
    WHERE survived = 1
    '''
)

avg(fare)
48.39540760233917


# Questão 6

In [23]:
spark.sql("SELECT COUNT(*) FROM train")

count(1)
891


# Questão 7

In [25]:
spark.sql('''
    SELECT
          (COUNT(CASE WHEN Sex = 'male' THEN 1 END) * 100 / COUNT(*)) AS PROPORCAO
    FROM train
''')

PROPORCAO
64.75869809203142


# Questão 8

In [34]:
spark.sql('''
    SELECT 
          SUM(CASE WHEN Pclass = 3 AND Survived = 1 THEN 1 END) * 100 / COUNT(CASE WHEN Pclass = 3 THEN 1 END) AS PROPORCAO
    FROM train
          ''')

PROPORCAO
24.236252545824847


# Questão 9

In [44]:
mall = spark.read.csv('../datasets/Mall_Customers.csv', header= True, inferSchema=True)

In [45]:
df_mall = mall.createOrReplaceTempView("mall")

spark.sql(
    '''
    SELECT AVG(Age) FROM mall
'''
)

avg(Age)
38.85


# Questão 10

In [46]:
spark.sql("SELECT COUNT(CASE WHEN Gender = 'Female' THEN 1 END ) FROM mall")

count(CASE WHEN (Gender = Female) THEN 1 END)
112


# Questão 11

In [94]:
media_renda_anual = mall.agg((F.avg('Annual Income (k$)')))
media_renda_anual

avg(Annual Income (k$))
60.56


In [51]:
from pyspark.sql.functions import avg
mall.select(
    avg("Annual Income (k$)").alias("media_renda_anual")
    )

media_renda_anual
60.56


In [53]:
spark.sql('''
    SELECT AVG(`Annual Income (k$)`) FROM mall
''')

avg(Annual Income (k$))
60.56


# Questão 12 

In [ ]:
mall.describe('Age')

In [89]:
from pyspark.sql.functions import stddev_pop, stddev

mall.select(stddev_pop(F.col('Age')))


stddev_pop(Age)
13.934041050606963


In [92]:
desv_idade = mall.agg(F.stddev_pop('Age'))
desv_idade

stddev_pop(Age)
13.934041050606963


In [61]:
spark.sql( '''
   SELECT STDDEV_POP(Age) AS desvio_padrao_populacional_idade FROM mall
''')

desvio_padrao_populacional_idade
13.934041050606963


# Questão 13

In [85]:
renda_80k_mais = mall.select(F.col('Annual Income (k$)')).where(F.col('Annual Income (k$)') > 80).count()
renda_80k_mais

38

In [86]:
spark.sql("SELECT COUNT(*) FROM mall WHERE `Annual Income (k$)` > 80")

count(1)
38


# Questão 14

In [79]:
spark.sql('''
    SELECT MODE(`Spending Score (1-100)`) FROM mall
''')

mode(Spending Score (1-100))
42


In [84]:
moda = mall.agg(F.mode('Spending Score (1-100)'))
moda

mode(Spending Score (1-100))
42
